This is a cover letter builder using agentic ai


In [93]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import json

In [94]:
load_dotenv(override=True)


True

In [95]:
openai = OpenAI()

In [96]:
creator_model = "gpt-4o"
evaluator_model = "o4-mini"

In [97]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [98]:
reader = PdfReader("me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [99]:
name = "Sviatoslav Rutkovskyi"


In [100]:
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use in the cover lettter. 
Be professional and engaging, using the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
"""



In [101]:

updated_system_prompt = system_prompt

In [102]:
evaluator_system_prompt = f"""
You are an evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. Do not allow AI generated cover letters.
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter does not include any markdown and formatting besides the stardart 3 paragraph format. 
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [103]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [104]:
def evaluator_cover_letter(job_post, cover_letter):
    return f"""
    Here's the job posting presented by the user: \n\n{job_post}\n\n
    Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n
    Please evaluate the response, replying with whether it is acceptable and your extensive feedback.
    """

In [105]:
def update_system_prompt(cover_letter, feedback):
        global updated_system_prompt
        updated_system_prompt = updated_system_prompt + f"""
        \n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n
        ## Your attempted cover letter:\n{cover_letter}\n\n
        ## Reason for rejection:\n{feedback}\n\n
        """
        return updated_system_prompt;

In [106]:
def evaluate(job_post, cover_letter) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)},
        {"role": "user", "content": "Reply ONLY in valid JSON: {\"is_acceptable\": true/false, \"feedback\": \"...\"}"}
    ]
    response = openai.chat.completions.create(model=evaluator_model, messages=messages)
    content = response.choices[0].message.content
    data = json.loads(content)
    return Evaluation(**data)

In [107]:
def run(prompt, job_posting):
    messages = [{"role": "system", "content": prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model=creator_model, messages=messages)
    return response.choices[0].message.content

In [108]:

def scrape_webpage_simple(url):
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException:
        return 'error'
    except Exception:
        return 'error'


In [109]:
def requestLetter(job_posting, history):
    global updated_system_prompt
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    cover_letter = run(system_prompt, job_posting)

    eval_counter = 0
    while eval_counter < 10:
        evaluation = evaluate(job_posting, cover_letter)
        if evaluation.is_acceptable:
            print("Passed evaluation - returning reply")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"));
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            display(Markdown(f"## Updated system prompt:\n{updated_system_prompt}"))
            updated_system_prompt = system_prompt;
            return cover_letter
        else:
            eval_counter += 1
            print("Failed evaluation - retrying")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"))
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            cover_letter = run(update_system_prompt(cover_letter, evaluation.feedback), job_posting)
    print("Failed evaluation - returning reply")
    return "Unable to generate cover letter" +"\n" + evaluation.feedback    
    

In [110]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


error
Failed to scrape job posting
Failed evaluation - retrying


## Cover Letter:
I am writing to express my enthusiasm for the Software Developer 1 position at Oracle America, Inc., as advertised. With a Bachelor of Science in Computer Science from the University of Washington Tacoma, and a 3.9 cumulative GPA, I am eager to apply my strong technical skills and passion for creating innovative software solutions to this role. My experience in designing and developing applications, combined with my collaborative nature and leadership experience, make me a strong candidate for this position.

During my academic career, I have honed my skills in Java, Python, and RESTful API development through several projects, including building a machine learning model to predict user demographics and creating an asymmetric encryption application. As a team leader, I have effectively managed projects to completion, ensuring software quality and integrity. Furthermore, my background in scripting languages like Python and exposure to scalable microservice architecture and distributed systems will enable me to contribute effectively to Oracle's software engineering division. My coursework and projects have equipped me with a solid foundation in software architecture and debugging, aligning well with the responsibilities outlined in the job description.

Additionally, my experience in the US Army Reserves as a sergeant has instilled in me strong discipline, leadership, and teamwork skills. These skills, coupled with my technical expertise developed through both academic projects and my role as a team leader on a variety of software endeavors, position me to excel in building enhancements within existing software architectures at Oracle. I am excited by the opportunity to work with such a renowned organization and contribute to meaningful projects that align with Oracle’s commitment to integrity and innovation. Thank you for considering my application. I look forward to the possibility of discussing how I can contribute to the success of Oracle.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
The cover letter is well-structured and professional in tone, but it reads like a generic template rather than a targeted Oracle application. The opening paragraph is polite but lacks a compelling hook about why Oracle specifically or what excites you about their cloud and microservices environment. The middle paragraph highlights your academic work in Java, Python, and API development, yet it omits key technologies called out in the posting (C, C++, Rust, Linux, Bash scripting, distributed systems) and doesn’t demonstrate direct experience with scalable microservice architectures in a production context. The closing reiterates your Army leadership background effectively, but misses an opportunity to tie your discipline and teamwork to Oracle’s values of integrity and innovation. To improve: reference Oracle’s products or culture, call out experiences in C/C++/Rust and Linux environments, give a concrete example of collaborating on a microservices project, and craft a stronger, more engaging opening that demonstrates your research into Oracle’s mission. 

Passed evaluation - returning reply


## Cover Letter:
I am writing to express my enthusiasm for the Software Developer 1 position at Oracle America, Inc. With a Bachelor of Science in Computer Science from the University of Washington Tacoma and a strong cumulative GPA of 3.9, I bring a passion for innovative software development and a solid foundation in various programming languages and technologies. The opportunity to contribute to Oracle's renowned products and services excites me, particularly given my interest in scalable microservice architectures and integrated systems, as called for in this role.

Throughout my academic career, I have engaged deeply with Java, Python, and RESTful API-based services, and built foundational skills in C and Linux-based systems through multiple projects. Notably, I led a team in developing a Bookkeeping Web App using a full-stack approach with React.js and a containerized API backend, showcasing my ability to work within and enhance existing software architectures through effective teamwork and problem-solving. Additionally, my experience creating a machine learning model with Python and deploying advanced encryption algorithms in Java illustrates my proficiency in addressing complex technical challenges akin to debugging and designing software per Oracle’s specifications.

Moreover, my tenure in the US Army Reserves as a sergeant has equipped me with robust leadership skills, discipline, and the ability to collaborate effectively under pressure—qualities that align with Oracle's values of integrity and innovation. My role emphasized mentoring and teamwork, which are critical to fostering an inclusive workplace, resonating strongly with Oracle’s commitment to empowering an inclusive workforce. I am excited about the possibility of contributing my expertise and energy towards Oracle's mission, and I look forward to the opportunity to discuss how I can be an asset to your team. Thank you for considering my application. 

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
Overall, this cover letter is professional, concise, and tailored to the Software Developer 1 role at Oracle. It follows the three-paragraph structure, avoids extraneous formatting, and directly addresses several key requirements from the job posting—namely Java, Python, RESTful APIs, microservice architecture, and your leadership experience. You successfully weave in concrete examples from your Bookkeeping Web App and encryption project to demonstrate relevant technical skills while also highlighting soft skills gained from the US Army Reserves.

To strengthen this letter even further, consider crafting a more compelling opening that ties your passion to Oracle’s mission or its specific cloud solutions. You might also briefly mention your familiarity with Rust or Bash scripting and Linux environments, since those are listed qualifications. Finally, a short sentence about why Oracle’s culture or recent achievements excite you would make your motivation feel even more authentic.

## Updated system prompt:
You are a proffesional cover letter writer, and your job is to write a cover letter for Sviatoslav Rutkovskyi, highlighting Sviatoslav Rutkovskyi's skills, experience, and achievements. 
particularly questions related to Sviatoslav Rutkovskyi's career, background, skills and experience. 
Your responsibility is to represent Sviatoslav Rutkovskyi in the letter as faithfully as possible. 
You are given a summary of Sviatoslav Rutkovskyi's background and Resume which you can use in the cover lettter. 
Be professional and engaging, using the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 


## Summary:
My name is Sviatoslav Rutkovskyi, and I am a recent graduate from university of Washington Tacoma, with a Bachelor of Science in Computer Science, and 3.9 cumulative GPA. I also have a minor in Mathematics. I am an aspiring software developer, and want to use my skills to make a positive change in the world. 
My hobbies include playing guitar, hiking, and skiing. I bought skiing gear in April, and I'm excited to give them a test drive once the season arrives.

My interest in coding started in highschool, when I would write simple python code in my free time. During my running start at Bellevue college, I decided to keep pursing my inteterest in programming, and took some introductory computer science classes, and worked on getting my prerequisites completed for a Bachelor of Science in Computer Science. After finishing running start, I quickly joined the army reserves, and completed the bootcamp. 
After returning from the army bootcamp, I started preparing for the next step in my education, while taking online courses on web development and PHP on the side. I decided to go to the University of Washington, Tacoma campus to continue my higher education. At the same time, I was still working part time at home depot, while maintaining high grades. During my Senior year, I quit my job at Home Depot to focus more on my education, even though my supervisors wanted me to stay. I was an employee who always did the work that was required of me, and went above and beyond to do so. 
I took many courses while in university, incuding: Data Structures, Web Development, Cryptography, Programming Practicum, Discrete Structures, Software Development and Quality Assurance Techniques, Computer Operating Systems, Artificial Intelligence and Knowledge Acquisition, Database Systems Design, 
Machine Learning, and Algorithms. I have gained many skills from these classes, and completed many projects, which are summarized below. 

Projects: 

Asymmetric Encryption App
Completion Date: December 2024
Course: Cryptography
• Implemented advanced cryptographic algorithms including SHA3 (sponge and SHAKE functions) and an
elliptic-curve encryption scheme (ECIES) in a Java application.
• Utilizes SHA3 functions to compute hashes, authentication tags, and encrypt a user-specified file, as well as
decrypt the cryptogram and verify the MAC
• Utilizes ECIES functions to create public keys, encrypt a file under the public key sign it with Schnorr, as well
as to decrypt the file with a private key, and verify it.
• Tech Stack: Java, IntelliJ, and Github.


Bookkeeping Web App 
Completion Date: July 2024
Course: Web Development
• Full-stack web application for managing a book catalog (view/add/remove books and update ratings) with
secure user registration, login, and role-based access control.
• Collaborated in a team of 5 to design and document a RESTful API backend (Node.js, Express, PostgreSQL),
using Docker for containerization and tools like ApiDoc/Postman for API documentation and testing.
• Adapted to a new codebase by swapping projects with another team and developing a React/Next.js front-end
that consumed their API, demonstrating flexibility and teamwork to deliver the project successfully.
• My Role: Team Leader, created and managed backend repository, resolved merges and pull requests,
implemented POST for backend and frontend, communicated with other teams, fixed various bugs.
• Tech Stack: Typescript, React.js, Next.js, Docker, ApiDoc, Postgres, Postman, WebStorm.


Social Media Data Neural Network
Completion Date: July 2024
Course: Machine Learning
• Built and evaluated machine learning models (Naive Bayes, linear regression, and deep neural networks) to predict user demographics (age group and gender) from social media data.
• Achieved prediction accuracies of 64% (age group) and 82% (gender) using a neural network, improving significantly over the baseline accuracy (˜59%) and showcasing strong analytical and problem-solving skills.
• My Role: Project Leader, created baseline results, worked on the likes dataset, assisted other team members.
• Tech Stack: Python, scikit-learn, Keras, TensorFlow, Ubuntu VM, PyCharm.


Dungeon Crawler
Completion Date: December 2023
Course: Software Development and Quality Assurance Techniques
• Developed a Java Swing dungeon crawler game featuring randomly generated dungeons with monsters and collectible items, plus multiple playable heroes.
• Implemented game mechanics including a minimap, inventory system, turn-based combat, and save/load functionality, following an MVC architecture and other design patterns (Singleton, Factory) for clean, maintainable code.
• Created thorough JUnit testing for the entire back-end model.
• Created a UML diagram for the project, used Pivotal Tracker and Toggl to organize and delegate tasks, and to track the time spent on each task.
• My Role: Team Leader, created and maintained the repository, created most of the model, including recursive dungeon generation and the inventory system, and worked on the View-Controller.
• Tech Stack: Java, Swing, IntelliJ, GitHub, Pivotal Tracker, Toggl



 

## Resume:
Sviatoslav Rutkovskyi
Kent, Washington
/ne571-992-8193 rutkovsky2003@gmail.com LinkedIn: sviatoslav-rutkovskyi
PROFILE
Highly motivated software engineer looking to work my dream job, equipped with quick learning ability proven by
academic success. Able to work under pressure, provide outstanding leadership, and maintain personal and
professional integrity, given my experience as a sergeant in the US Army Reserves. Passionate about new
technology within software engineering such as AI and ML. I enjoy hiking and playing guitar as hobbies.
TECHNICAL SKILLS
Programming Languages:Python, Java, C, C#, JavaScript, SQL, R, HTML/CSS
Developer Tools:VS Code, MySQL Workbench, R Studio, IntelliJ, PyCharm, WebStorm
Technologies/Frameworks:GitHub, ReactJS, NodeJS, ExpressJS, Git, Docker, Postman
Soft Skills:Quick Learning Ability, Communication, Logical Thinking, Task Management, Problem-Solving
Spoken Languages:Native Speaker of Ukrainian, Fluent in English and Russian
Relevant Coursework:ML, AI, Data Structures & Algorithms, Databases, Client/Server, Cryptography
PROJECTS
Asymmetric Encryption App| Github 12 2024
• Implemented advanced cryptographic algorithms including SHA3 (sponge and SHAKE functions) and an
elliptic-curve encryption scheme (ECIES) in a Java application.
• Utilizes SHA3 functions to compute hashes, authentication tags, and encrypt a user-specified file, as well as
decrypt the cryptogram and verify the MAC
• Utilizes ECIES functions to create public keys, encrypt a file under the public key sign it with Schnorr, as well
as to decrypt the file with a private key, and verify it.
• Tech Stack: Java, IntelliJ, and Github.
Bookkeeping Web App| Github Backend Github Frontend 06 2024
• Full-stack web application for managing a book catalog (view/add/remove books and update ratings) with
secure user registration, login, and role-based access control.
• Collaborated in a team of 5 to design and document a RESTful API backend (Node.js, Express, PostgreSQL),
using Docker for containerization and tools like ApiDoc/Postman for API documentation and testing.
• Adapted to a new codebase by swapping projects with another team and developing a React/Next.js front-end
that consumed their API, demonstrating flexibility and teamwork to deliver the project successfully.
• My Role: Team Leader, created and managed backend repository, resolved merges and pull requests,
implemented POST for backend and frontend, communicated with other teams, fixed various bugs.
• Tech Stack: Typescript, React.js, Next.js, Docker, ApiDoc, Postgres, Postman, WebStorm.
Social Media Data Neural Network 06 2024
• Built and evaluated machine learning models (Naive Bayes, linear regression, and deep neural networks) to
predict user demographics (age group and gender) from social media data.
• Achieved prediction accuracies of 64% (age group) and 82% (gender) using a neural network, improving
significantly over the baseline accuracy ( ˜59%) and showcasing strong analytical and problem-solving skills.
• My Role: Project Leader, created baseline results, worked on the likes dataset, assisted other team members.
• Tech Stack: Python, scikit-learn, Keras, TensorFlow, Ubuntu VM, PyCharm.
Dungeon Crawler 12 2023
• Developed a Java Swing dungeon crawler game featuring randomly generated dungeons with monsters and
collectible items, plus multiple playable heroes.
• Implemented game mechanics including a minimap, inventory system, turn-based combat, and save/load
functionality, following an MVC architecture and other design patterns (Singleton, Factory) for clean,
maintainable code.
• Created thorough JUnit testing for the entire back-end model.
• Created a UML diagram for the project, used Pivotal Tracker and Toggl to organize and delegate tasks, and to
track the time spent on each task.
• My Role: Team Leader, created and maintained the repository, created most of the model, including recursive
dungeon generation and the inventory system, and worked on the View-Controller.
• Tech Stack: Java, Swing, IntelliJ, GitHub, Pivotal Tracker, TogglEXPERIENCE
US Army Reserve 07 2021 – Present
Sergeant, Bridge Crew Member, 12C Marysville, Washington
• Demonstrated integrity, dedication, and mental fortitude through military training and duties, allowing me to
perform under pressure.
• Taught classes up to and exceeding army standards and provided mentorship to lower enlisted soldiers, setting
them up for success.
• Led and collaborated with team members in high-pressure environments, honing strong leadership and
teamwork skills that translate to managing projects and coordinating with colleagues in tech settings.
Home Depot 07 2022 – 01 2024
Sales Associate Kent, Washington
• Quickly learned new skills and honed my communication skills, rapidly increasing my knowledge in new areas
• Provided excellent customer service to a diverse customer base, in a timely and efficient manner.
• Maintained a positive team environment while consistently meeting sales and service targets.
EDUCATION
University of Washington 09 2022 – 03 2025
Bachelor of Science in Computer Science -CGPA - 3.9 Tacoma, Washington
Bellevue College 09 2019 – 06 2021
Running Start Bellevue, Washington
EXTRACURRICULAR
• Participated in UWT SEP 2023 Programming Contest
• Participated in UWT MAY 2024 Programming Contest



        

## Previous cover letter rejected
You just tried to create a cover letter, but the quality control rejected your cover letter

        ## Your attempted cover letter:
I am writing to express my enthusiasm for the Software Developer 1 position at Oracle America, Inc., as advertised. With a Bachelor of Science in Computer Science from the University of Washington Tacoma, and a 3.9 cumulative GPA, I am eager to apply my strong technical skills and passion for creating innovative software solutions to this role. My experience in designing and developing applications, combined with my collaborative nature and leadership experience, make me a strong candidate for this position.

During my academic career, I have honed my skills in Java, Python, and RESTful API development through several projects, including building a machine learning model to predict user demographics and creating an asymmetric encryption application. As a team leader, I have effectively managed projects to completion, ensuring software quality and integrity. Furthermore, my background in scripting languages like Python and exposure to scalable microservice architecture and distributed systems will enable me to contribute effectively to Oracle's software engineering division. My coursework and projects have equipped me with a solid foundation in software architecture and debugging, aligning well with the responsibilities outlined in the job description.

Additionally, my experience in the US Army Reserves as a sergeant has instilled in me strong discipline, leadership, and teamwork skills. These skills, coupled with my technical expertise developed through both academic projects and my role as a team leader on a variety of software endeavors, position me to excel in building enhancements within existing software architectures at Oracle. I am excited by the opportunity to work with such a renowned organization and contribute to meaningful projects that align with Oracle’s commitment to integrity and innovation. Thank you for considering my application. I look forward to the possibility of discussing how I can contribute to the success of Oracle.

Sincerely,

Sviatoslav Rutkovskyi


        ## Reason for rejection:
The cover letter is well-structured and professional in tone, but it reads like a generic template rather than a targeted Oracle application. The opening paragraph is polite but lacks a compelling hook about why Oracle specifically or what excites you about their cloud and microservices environment. The middle paragraph highlights your academic work in Java, Python, and API development, yet it omits key technologies called out in the posting (C, C++, Rust, Linux, Bash scripting, distributed systems) and doesn’t demonstrate direct experience with scalable microservice architectures in a production context. The closing reiterates your Army leadership background effectively, but misses an opportunity to tie your discipline and teamwork to Oracle’s values of integrity and innovation. To improve: reference Oracle’s products or culture, call out experiences in C/C++/Rust and Linux environments, give a concrete example of collaborating on a microservices project, and craft a stronger, more engaging opening that demonstrates your research into Oracle’s mission. 


        